# Detectar entidades y sentimientos en un documento 🔎 📄.

En este notebook podrás extraer texto de una imagen, identificar los sentimientos  y además las identidades mencionadas en el texto.

![Detecting entities and sentiment from a document"](../images/detecting-entities-and-sentiment.png)

### Este ejemplo vas a usar 2 servicios de AI/ML que puedes emplear como llamadas de API:  

### [Amazon Comprehend](https://aws.amazon.com/comprehend/): 
Utiliza el procesamiento del lenguaje natural (NLP) para extraer información sobre el contenido de los documentos. Desarrolla información mediante el reconocimiento de las entidades, las frases clave, el **idioma**, los sentimientos y otros elementos comunes de un documento. Utilice Amazon Comprehend para crear nuevos productos basándose en la comprensión de la estructura de los documentos. Por ejemplo, con Amazon Comprehend puedes buscar menciones de productos en las redes sociales o escanear todo un repositorio de documentos en busca de frases clave. [Más información](https://docs.aws.amazon.com/comprehend/latest/dg/what-is.html)


### [Amazon Textract](https://docs.aws.amazon.com/textract/latest/dg/what-is.html): 
Detección y el análisis del texto de los documentos a sus aplicaciones. 
Con Amazon Textract, puede hacer lo siguiente:

- Detecte texto mecanografiado y manuscrito en una variedad de documentos, incluidos informes financieros, registros médicos y formularios de impuestos.
- Extraiga texto, formularios y tablas de documentos con datos estructurados mediante la API de análisis de documentos de Amazon Textract.
- Especifique y extraiga información de los documentos mediante la función de consultas de la API Amazon Textract Analyze Document.
- Procese facturas y recibos con la API AnalyzeExpense.
- Procese documentos de identificación, como licencias de conducir y pasaportes emitidos por el gobierno de EE. UU., mediante la API AnalyzeID.

Cargue y procese paquetes de préstamos hipotecarios mediante el direccionamiento automático de las páginas del documento a las operaciones de análisis de Amazon Textract correspondientes mediante el flujo de trabajo Analyze Lending. Puede recuperar los resultados del análisis de cada página del documento o puede recuperar los resultados resumidos de un conjunto de páginas del documento.

### Requisitos:
- [Una cuenta de AWS](https://aws.amazon.com/es/free/)
- [Foundational knowledge of Python](https://community.aws/tutorials/build-a-knowledge-base-with-multilingual-q-and-a-gen-ai)

### Configuración previa:
- [Uso de AWS Command Line Interface CLI](https://docs.aws.amazon.com/es_es/cli/latest/userguide/cli-chap-welcome.html)
- [Instalar CLI](https://docs.aws.amazon.com/es_es/cli/latest/userguide/getting-started-install.html)
- [Configurar credenciales de AWS para uso local](https://docs.aws.amazon.com/es_es/cli/latest/userguide/cli-configure-files.html)

## Empecemos!! 🔎 📄.

### Paso 0: [Instala Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#install-boto3) [Más información de Boto3](https://aws.amazon.com/es/sdk-for-python/)
Es una librería de python para facilitar la integración con los servicios de AWS (Amazon Web Services)

### Instala la ultima version de Boto3 via pip: 

!pip install boto3

In [16]:
# Boto3 session with credentials taken from environtment
import boto3
import os

AWS_ACCESS_KEY_ID = YOU_AWS_ACCESS_KEY_ID
AWS_SECRET_ACCESS_KEY = YOU_AWS_SECRET_ACCESS_KEY
AWS_SESSION_TOKEN= YOU_AWS_SESSION_TOKEN
REGION_NAME = YOU_REGION

my_session = boto3.Session(
    aws_access_key_id     = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
    aws_session_token     = AWS_SESSION_TOKEN,
    region_name=REGION_NAME
)

### Paso 1: Detecta el texto en el documento

> 🚨Nota: Cada servicio tiene un cliente que debes inicializar antes de hacer cada llamada de API. 

Este paso lo realizamos empleando la llamda de API [Detect Document Text API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/textract/client/detect_document_text.html) de [Boto3 Textract client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/textract.html).

In [17]:
# Amazon Textract client
textract_client = my_session.client('textract')

def extraer_texto(document):
    response = textract_client.detect_document_text(
            Document={
                'Bytes': document.read(),
            }
        )
    return response

In [14]:
# Document
documentName = "simple-document-image.jpg"
with open(documentName, "rb") as document:
    response = extraer_texto(document)
response

{'DocumentMetadata': {'Pages': 1},
 'Blocks': [{'BlockType': 'PAGE',
   'Geometry': {'BoundingBox': {'Width': 1.0,
     'Height': 1.0,
     'Left': 0.0,
     'Top': 0.0},
    'Polygon': [{'X': 0.0, 'Y': 0.0},
     {'X': 1.0, 'Y': 0.0},
     {'X': 1.0, 'Y': 1.0},
     {'X': 0.0, 'Y': 1.0}]},
   'Id': '096286fb-80e9-4ccb-bd80-4e44289c7ae2',
   'Relationships': [{'Type': 'CHILD',
     'Ids': ['ea17d570-138a-4471-8956-157254a32447',
      'e5652fd4-9560-4a30-94f9-16e278b9600e',
      'e81ee186-1daa-4ea1-bd82-b5446ecaab03',
      '2013c2ee-663c-44e5-9181-415ec438df09']}]},
  {'BlockType': 'LINE',
   'Confidence': 99.52398681640625,
   'Text': 'Amazon.com, Inc. is located in Seattle, WA',
   'Geometry': {'BoundingBox': {'Width': 0.512660026550293,
     'Height': 0.06824082136154175,
     'Left': 0.06333211064338684,
     'Top': 0.1989629715681076},
    'Polygon': [{'X': 0.06337157636880875, 'Y': 0.20793944597244263},
     {'X': 0.5759921669960022, 'Y': 0.1989629715681076},
     {'X': 0.57596

In [15]:
# Para codificar el texto
print("El texto es:\n========")
text = ""
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print ('\033[94m' +  item["Text"] + '\033[0m')
        text = text + " " + item["Text"]

El texto es:
Amazon.com, Inc. is located in Seattle, WA
It was founded July 5th, 1994 by Jeff Bezos
Amazon.com allows customers to buy everything from books to blenders
Seattle is north of Portland and south of Vancouver, BC.


### Paso 2: Detecta los sentimientos en el documento


Este paso lo realizamos empleando la llamda de API [Detect Sentiment API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend/client/detect_sentiment.html) de [Boto3 Comprehend client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html).

In [ ]:
# Amazon Comprehend client
comprehend_client = my_session.client('comprehend')

sentiment =  comprehend_client.detect_sentiment(LanguageCode="en", Text=text)

print ("\nSentimentos\n========\n{}".format(sentiment.get('Sentiment')))




### Paso 3: Detecta los entidades en el documento


Este paso lo realizamos empleando la llamda de API [Detect entities API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend/client/detect_entities.html) de [Boto3 Comprehend client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend.html).

In [ ]:
entities =  comprehend_client.detect_entities(LanguageCode="en", Text=text)

print("\nEntidades\n========")

for entity in entities["Entities"]:
    print ("{}\t=>\t{}".format(entity["Type"], entity["Text"]))

> 👩🏻‍💻 **Tips:** Aprende mas de Amazon Transcribe y Amazon Comprehend en estos [Code Samples](https://github.com/aws-samples/amazon-transcribe-comprehend-podcast)

## ¡Gracias!

Te dejo mis redes: 
🇻🇪🇨🇱 [Dev.to](https://dev.to/elizabethfuentes12) [Linkedin](https://www.linkedin.com/in/lizfue/) [GitHub](https://github.com/elizabethfuentes12/) [Twitter](https://twitter.com/elizabethfue12) [Instagram](https://www.instagram.com/elifue.tech) [Youtube](https://www.youtube.com/channel/UCr0Gnc-t30m4xyrvsQpNp2Q)
[linktr](https://linktr.ee/elizabethfuentesleone)